In [1]:
import copy
import datetime as dt
import importlib
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import pathlib
import sys
import time
import pickle
from typing import Union
import warnings
from datetime import datetime
from pyswarm import pso  # Install with: pip install pyswarm
warnings.filterwarnings("ignore")
from utilities.utils import HelperFunctions, SSPModelForCalibration, SectoralDiffReport, NonEnergySectoralDiffReport, ErrorFunctions
import logging
from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Record the start time
start_time = time.time()

# Initialize helper functions
helper_functions = HelperFunctions()

In [4]:
# Paths
SRC_FILE_PATH = os.getcwd()
build_path = lambda PATH: os.path.abspath(os.path.join(*PATH))
DATA_PATH = build_path([SRC_FILE_PATH, "..", "data"])
OUTPUT_PATH = build_path([SRC_FILE_PATH, "..", "output"])
PSO_OUTPUT_PATH = build_path([OUTPUT_PATH, "pso"])
REAL_DATA_FILE_PATH = build_path([DATA_PATH, "real_data.csv"])
MISC_FILES_PATH = build_path([SRC_FILE_PATH, 'misc'])
MAPPING_FILES_PATH = build_path([MISC_FILES_PATH, 'mapping'])
SECTORAL_REPORT_PATH = build_path([MISC_FILES_PATH, 'sectoral_reports'])
OPT_CONFIG_FILES_PATH = build_path([SRC_FILE_PATH, 'config'])
OPT_OUTPUT_PATH = build_path([SRC_FILE_PATH,"..", "output"])


In [5]:
# Make sure the output directory exists
os.makedirs(OPT_OUTPUT_PATH, exist_ok=True)

In [6]:
# Get important params from the YAML file

try:
    yaml_file = 'croatia_opt_config.yaml'
except IndexError:
    raise ValueError("YAML configuration file must be provided as a command-line argument.")

param_dict = helper_functions.get_parameters_from_yaml(build_path([OPT_CONFIG_FILES_PATH, yaml_file]))

target_region = param_dict['target_region']
iso_alpha_3 = param_dict['iso_alpha_3']
stressed_variables_report_version = param_dict['stressed_variables_report_version']
normalization_flag = param_dict['normalization_flag']
detailed_diff_report_flag = param_dict['detailed_diff_report_flag']
energy_model_flag = param_dict['energy_model_flag']
error_type = param_dict['error_type'] 
unique_id = datetime.now().strftime("%Y%m%d%H%M%S")
swarm_size = param_dict['swarmsize']
maxiter = param_dict['maxiter']
input_rows = param_dict['input_rows']

logging.info(f"Starting optimization for {target_region} (ISO code: {iso_alpha_3})")
logging.info(f"Input rows: {input_rows}")
logging.info(f"Stressed variables report version: {stressed_variables_report_version}")
logging.info(f"Normalization flag: {normalization_flag}")
logging.info(f"Energy model flag: {energy_model_flag}")
logging.info(f"Error type: {error_type}")
logging.info(f"Unique ID: {unique_id}")
logging.info(f"Swarm size: {swarm_size}")
logging.info(f"Max iterations: {maxiter}")
logging.info(f"Detailed diff report flag: {detailed_diff_report_flag}")

2025-02-12 11:42:49,449 - INFO - Starting optimization for croatia (ISO code: HRV)


2025-02-12 11:42:49,450 - INFO - Input rows: 5
2025-02-12 11:42:49,451 - INFO - Stressed variables report version: stressed_variables_report_2025_02_07.xlsx
2025-02-12 11:42:49,452 - INFO - Normalization flag: False
2025-02-12 11:42:49,453 - INFO - Energy model flag: False
2025-02-12 11:42:49,453 - INFO - Error type: rmse
2025-02-12 11:42:49,455 - INFO - Unique ID: 20250212114249
2025-02-12 11:42:49,455 - INFO - Swarm size: 3
2025-02-12 11:42:49,456 - INFO - Max iterations: 2
2025-02-12 11:42:49,456 - INFO - Detailed diff report flag: True


In [7]:
# Make sure the output directory exists
os.makedirs(OPT_OUTPUT_PATH, exist_ok=True)

# Make sure pso output directories exist
os.makedirs(PSO_OUTPUT_PATH, exist_ok=True)

# Create the output directory for the PSO results using the unique ID
RUN_OUTPUT_DIR = os.path.join(PSO_OUTPUT_PATH, unique_id)
os.makedirs(RUN_OUTPUT_DIR, exist_ok=True)

In [8]:
# Save the config file to the output directory
config_file_name = f"{unique_id}_config.yaml"
config_file_output_path = os.path.join(RUN_OUTPUT_DIR, config_file_name)
helper_functions.copy_param_yaml(build_path([OPT_CONFIG_FILES_PATH, yaml_file]), config_file_output_path)

In [9]:
# Load input dataset
examples = SISEPUEDEExamples()
cr = examples("input_data_frame")
df_input = pd.read_csv(REAL_DATA_FILE_PATH)

# Add missing columns and reformat the input datas
df_input = df_input.rename(columns={'period': 'time_period'})
df_input = helper_functions.add_missing_cols(cr, df_input.copy())
df_input = df_input.drop(columns='iso_code3')

# Subset df_input to the input rows amount
df_input = df_input.iloc[:input_rows]

df_input

,region,time_period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,croatia,0,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,croatia,1,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,croatia,2,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,croatia,3,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,croatia,4,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


In [10]:
# Load frac_vars mapping excel
frac_vars_mapping = pd.read_excel(build_path([MAPPING_FILES_PATH, 'frac_vars_mapping.xlsx']), sheet_name='frac_vars')
frac_vars_mapping

,frac_var_name,frac_var_name_prefix,special_case
0,frac_agrc_agriculture_production_lost,frac_agrc_agriculture_production,1
1,frac_agrc_bevs_and_spices_cl1_temperate,frac_agrc_bevs_and_spices_cl1,0
2,frac_agrc_bevs_and_spices_cl1_tropical,frac_agrc_bevs_and_spices_cl1,0
3,frac_agrc_bevs_and_spices_cl2_dry,frac_agrc_bevs_and_spices_cl2,0
4,frac_agrc_bevs_and_spices_cl2_wet,frac_agrc_bevs_and_spices_cl2,0
...,...,...,...
817,frac_waso_recycled_paper,frac_waso_recycled,0
818,frac_waso_recycled_plastic,frac_waso_recycled,0
819,frac_waso_recycled_rubber_leather,frac_waso_recycled,0
820,frac_waso_recycled_textiles,frac_waso_recycled,0


In [11]:
# Load the stressed variables mapping file
stressed_vars_mapping = pd.read_excel(build_path([MAPPING_FILES_PATH, stressed_variables_report_version]))
stressed_vars_mapping

,variable_name,subsector,is_stressed,l_bound,u_bound,group_id,needs_check,is_capped,is_asymmetric
0,area_gnrl_country_ha,SocioEconomic,0,1.0,1.0,NaN,NaN,0,0
1,avgload_trns_freight_tonne_per_vehicle_aviation,Energy,1,0.5,1.5,0.0,NaN,0,0
2,avgload_trns_freight_tonne_per_vehicle_rail_fr...,Energy,1,0.5,1.5,0.0,NaN,0,0
3,avgload_trns_freight_tonne_per_vehicle_road_he...,Energy,1,0.5,1.5,0.0,NaN,0,0
4,avgload_trns_freight_tonne_per_vehicle_water_b...,Energy,1,0.5,1.5,0.0,NaN,0,0
...,...,...,...,...,...,...,...,...,...
2211,yf_agrc_pulses_tonne_ha,AFOLU,0,0.5,1.5,NaN,NaN,0,0
2212,yf_agrc_rice_tonne_ha,AFOLU,0,0.5,1.5,NaN,NaN,0,0
2213,yf_agrc_sugar_cane_tonne_ha,AFOLU,0,0.5,1.5,NaN,NaN,0,0
2214,yf_agrc_tubers_tonne_ha,AFOLU,0,0.5,1.5,NaN,NaN,0,0


In [12]:
# Subset the stressed variables mapping file to is_stressed = 1
stressed_vars_mapping = stressed_vars_mapping[stressed_vars_mapping['is_stressed'] == 1]

# Check for nulls in the is_stressed column
if stressed_vars_mapping['is_stressed'].isnull().sum() > 0:
    raise ValueError("There are null values in the is_stressed column of the stressed variables mapping file.")

In [13]:
# Reset the index of the stressed variables mapping file
stressed_vars_mapping = stressed_vars_mapping.reset_index(drop=True)

# Set group_id as integer
stressed_vars_mapping['group_id'] = stressed_vars_mapping['group_id'].astype(int)

In [14]:
# Check group id array
stressed_vars_mapping.group_id.unique()

array([  0,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
         1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
       109,  14,  15,  16,  19,  17,  18,  20,  21,  22, 110,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  40,  41,  39,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,  52, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,  53,  54,
       172, 173, 174, 175, 176, 177, 178, 179,  55,  56, 180, 181,  57,
       182,  58,  59,  60,  61,  62,  63, 183,  64, 184, 185, 18

In [15]:
# Get the list of vars to clip
vars_to_clip = stressed_vars_mapping[stressed_vars_mapping['is_capped'] == 1]['variable_name'].tolist()
# vars_to_clip

In [16]:
# Get the frac_vars that are going to be stressed
frac_vars_to_stress = [var for var in stressed_vars_mapping['variable_name'].values if var.startswith('frac_')]
# frac_vars_to_stress

In [17]:
# Subset frac_vars_mapping to only include the frac_vars that are going to be stressed
frac_vars_mapping = frac_vars_mapping[frac_vars_mapping['frac_var_name'].isin(frac_vars_to_stress)].reset_index(drop=True)
frac_vars_mapping

,frac_var_name,frac_var_name_prefix,special_case
0,frac_agrc_agriculture_production_lost,frac_agrc_agriculture_production,1
1,frac_agrc_dry_matter_of_harvest_cereals,frac_agrc_dry_matter_of_harvest,0
2,frac_agrc_dry_matter_of_harvest_other_annual,frac_agrc_dry_matter_of_harvest,0
3,frac_agrc_dry_matter_of_harvest_pulses,frac_agrc_dry_matter_of_harvest,0
4,frac_agrc_dry_matter_of_harvest_rice,frac_agrc_dry_matter_of_harvest,0
...,...,...,...
95,frac_waso_recycled_paper,frac_waso_recycled,0
96,frac_waso_recycled_plastic,frac_waso_recycled,0
97,frac_waso_recycled_rubber_leather,frac_waso_recycled,0
98,frac_waso_recycled_textiles,frac_waso_recycled,0


In [18]:
# Check special_case distribution
frac_vars_mapping['special_case'].value_counts()

special_case
0    94
1     6
Name: count, dtype: int64

In [19]:
# Get group ids of the vars that are stressed
group_ids = stressed_vars_mapping[stressed_vars_mapping["is_stressed"] == 1]["group_id"].unique()
n_groups = len(group_ids)

# Debug prints
print('Max group id: ', group_ids.max())
print('Total groups: ', n_groups)
print('Array dtype: ', type(group_ids[0]))

Max group id:  218
Total groups:  219
Array dtype:  <class 'numpy.int64'>


In [20]:
# Get the lower and upper bounds for each group
l_bounds = stressed_vars_mapping.groupby("group_id")["l_bound"].first().values
u_bounds = stressed_vars_mapping.groupby("group_id")["u_bound"].first().values

print(f"{10*'#'}bounds arrays{10*'#'}")
print(l_bounds)
print(u_bounds)

print(f"{10*'#'}bounds vector size{10*'#'}")
print('l_bounds:', len(l_bounds))
print('u_bounds:', len(u_bounds))

##########bounds arrays##########
[0.5  0.6  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.8  0.8  0.8  0.8  0.8  0.8
 0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.5  0.75 0.8  0.7
 0.75 0.75 0.8  0.75 0.75 0.75 0.75 0.8  0.75 0.8  0.8  0.95 0.75 0.75
 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75
 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75
 0.75 0.75 0.75 0.75 0.75 0.75 0.5  0.6  0.6  0.6  0.6  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.5  0.5  0.5  0.5  0.75 0.75 0.75 0.75 0.75 0.95 0.75 0.75 0.7  0.75
 0.75 0.5  0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0

In [21]:
# Create a dictionary with the group ids as keys and the corresponding variable names as values
group_vars_dict = {}
for group_id in group_ids:
    group_vars_dict[group_id] = stressed_vars_mapping[stressed_vars_mapping["group_id"] == group_id]["variable_name"].values

group_vars_dict

{0: array(['avgload_trns_freight_tonne_per_vehicle_aviation',
        'avgload_trns_freight_tonne_per_vehicle_rail_freight',
        'avgload_trns_freight_tonne_per_vehicle_road_heavy_freight',
        'avgload_trns_freight_tonne_per_vehicle_water_borne'], dtype=object),
 71: array(['consumpinit_inen_energy_tj_per_mmm_gdp_other_product_manufacturing'],
       dtype=object),
 72: array(['consumpinit_inen_energy_tj_per_tonne_production_cement'],
       dtype=object),
 73: array(['consumpinit_inen_energy_tj_per_tonne_production_chemicals'],
       dtype=object),
 74: array(['consumpinit_inen_energy_tj_per_tonne_production_electronics'],
       dtype=object),
 75: array(['consumpinit_inen_energy_tj_per_tonne_production_glass'],
       dtype=object),
 76: array(['consumpinit_inen_energy_tj_per_tonne_production_lime_and_carbonite'],
       dtype=object),
 77: array(['consumpinit_inen_energy_tj_per_tonne_production_metals'],
       dtype=object),
 78: array(['consumpinit_inen_energy_tj_per_to

In [22]:
# Initialize class instances
if energy_model_flag:
    diff_report_helpers = SectoralDiffReport(SECTORAL_REPORT_PATH, iso_alpha_3, init_year=2015)
else:
    diff_report_helpers = NonEnergySectoralDiffReport(SECTORAL_REPORT_PATH, iso_alpha_3, init_year=2015)
ef = ErrorFunctions()

In [23]:
# Initialize global variable to store the previous calculated error
previous_error = float('inf')

# Initialize global variable to store the worst_valid_error
worst_valid_error = float(2)

In [24]:
# Initialize the SSP model
ssp_model = SSPModelForCalibration(energy_model_flag=energy_model_flag)

2025-02-12 11:42:52,029 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-02-12 11:42:52,029 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-02-12 11:42:52,033 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-02-12 11:42:52,033 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-02-12 11:42:52,034 - INFO - 	Setting export engine to 'csv'.
2025-02-12 11:42:52,034 - INFO - 	Setting export engine to 'csv'.
2025-02-12 11:42:52,036 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2025-02-12 11:42:52,036 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2025-02-12 11:42:52,038 - INFO - Successfully instantiated table ANALYSIS_METADATA
2025-02-12 11:42:52,038 - INFO - Successfully

In [25]:
# Simulation model
def simulation_model(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function that simulates outputs based on the scaled inputs.
    """
    sim_output_df = ssp_model.run_ssp_simulation(df)
    
    # Handle empty DataFrame
    if sim_output_df is None or sim_output_df.empty:
        logging.warning("Simulation Output DataFrame is empty. Returning an empty DataFrame.")
        return pd.DataFrame()

    return sim_output_df


In [26]:
# Define the objective function
def objective_function(x):
    
    # Global variables
    global worst_valid_error
    global previous_error
    
    
    # x: scaling factors for each group_id
    modified_df = df_input.copy()
    
    # TODO: Vectorize this loop
    # Scale the variables per group
    for group_id in group_vars_dict:
        for var in group_vars_dict[group_id]:
            modified_df[var] = modified_df[var] * x[group_id]
    
    
    if normalization_flag:
        # Handle frac var group normalization
        processed_input_df = helper_functions.simple_frac_normalization(modified_df, frac_vars_mapping)

        # Clip the variables
        processed_input_df = helper_functions.clip_values(processed_input_df, vars_to_clip)
    
    else:
        processed_input_df = modified_df.copy()

    
    # Run the model
    sim_output_df = simulation_model(processed_input_df)
    
    # Assing a penalty if the simulation output is empty
    if sim_output_df.empty:
        error_val = worst_valid_error * 1.1  # Slighly higher than the worst valid error for invalid outputs
        logging.warning("Simulation returned an empty DataFrame. Setting Error to a penalty value.")
    
    else:
        # Generate diff reports to calculate Error
        detailed_diff_report, subsector_diff_report = diff_report_helpers.run_report_generator(simulation_df=sim_output_df)

        
        # Checks if the model failed in any subsector
        if diff_report_helpers.model_failed_flag:
            error_val = worst_valid_error * 1.1  # Slighly higher than the worst valid error for invalid outputs
            logging.warning("Model failed in a subsector. Setting Error to a penalty.")
        
        # Calculate error: Subsectors with Edgar value == 0.0 are not considered
        elif detailed_diff_report_flag:
            error_val = ef.calculate_error(error_type, detailed_diff_report)
        else:
            error_val = ef.calculate_error(error_type, subsector_diff_report)

    # Update worst_valid_error
    if error_val > worst_valid_error:
        worst_valid_error = error_val
        logging.info(f"New worst_valid_error: {worst_valid_error:.6f}")

    # Log the error
    logging.info("=" * 30)
    logging.info(f"Current ERROR: {error_val:.6f}")
    logging.info("=" * 30)

    # Log the scaling factors and the error
    helper_functions.log_to_csv(x, error_val, error_type, RUN_OUTPUT_DIR, target_region, unique_id)

    # Save the processed_input_df, detailed_diff_report and subsector_diff_report if the error is less than the previous error
    if error_val < previous_error:
        previous_error = error_val
        processed_input_df.to_csv(build_path([RUN_OUTPUT_DIR, f"best_input_df_{unique_id}.csv"]), index=False)
        detailed_diff_report.to_csv(build_path([RUN_OUTPUT_DIR, f"best_detailed_diff_report_{unique_id}.csv"]), index=False)
        subsector_diff_report.to_csv(build_path([RUN_OUTPUT_DIR, f"best_subsector_diff_report_{unique_id}.csv"]), index=False)
        logging.info(f"Best Input Data and Diff Reports Updated to {RUN_OUTPUT_DIR}")

    return error_val

In [27]:
# Initialize the PSO optimizer
best_solution, best_value = pso(
    objective_function,
    l_bounds,
    u_bounds,
    swarmsize=swarm_size,
    maxiter=maxiter,
    )

2025-02-12 11:43:37,062 - INFO - Running AFOLU model
2025-02-12 11:43:37,062 - INFO - Running AFOLU model
2025-02-12 11:43:37,206 - INFO - AFOLU model run successfully completed
2025-02-12 11:43:37,206 - INFO - AFOLU model run successfully completed
2025-02-12 11:43:37,208 - INFO - Running CircularEconomy model
2025-02-12 11:43:37,208 - INFO - Running CircularEconomy model
2025-02-12 11:43:37,270 - INFO - CircularEconomy model run successfully completed
2025-02-12 11:43:37,270 - INFO - CircularEconomy model run successfully completed
2025-02-12 11:43:37,272 - INFO - Running IPPU model
2025-02-12 11:43:37,272 - INFO - Running IPPU model
2025-02-12 11:43:37,359 - INFO - IPPU model run successfully completed
2025-02-12 11:43:37,359 - INFO - IPPU model run successfully completed
2025-02-12 11:43:37,360 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2025-02-12 11:43:37,360 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2025-02-12 1

Stopping search: maximum iterations reached --> 2


In [28]:
# logging.info(f"Best scaling vector: {best_solution}")
logging.info(f"Best error: {best_value}")

2025-02-12 11:43:43,611 - INFO - Best error: 1.3165443794533431
